In [35]:
import pandas as pd
import numpy as np
import random
import os

from sklearn.preprocessing import LabelEncoder
import pandas as pd
train = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')

In [49]:
train

,ID,월,요일,시간,소관경찰서,소관지역,사건발생거리,강수량(mm),강설량(mm),적설량(cm),풍향,안개,짙은안개,번개,진눈깨비,서리,연기/연무,눈날림,범죄발생지,TARGET
0,TRAIN_00000,9,화요일,10,137,8.0,2.611124,0.000000,0.0,0.00,245.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,차도,2
1,TRAIN_00001,11,화요일,6,438,13.0,3.209093,0.000000,0.0,0.00,200.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,차도,0
2,TRAIN_00002,8,일요일,6,1729,47.0,1.619597,0.000000,0.0,0.00,40.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,인도,1
3,TRAIN_00003,5,월요일,6,2337,53.0,1.921615,11.375000,0.0,0.00,225.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,주거지,1
4,TRAIN_00004,9,일요일,11,1439,41.0,1.789721,0.000000,0.0,0.00,255.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,주유소,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84401,TRAIN_84401,4,일요일,7,336,11.0,3.808190,99.111111,0.0,0.00,165.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,차도,1
84402,TRAIN_84402,8,목요일,12,2149,38.0,1.458490,0.000000,0.0,0.00,200.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,차도,0
84403,TRAIN_84403,7,일요일,6,29,46.0,2.944913,105.888889,0.0,0.00,315.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,차도,0
84404,TRAIN_84404,1,화요일,11,536,25.0,0.493679,2.285714,8.6,10.75,330.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,주거지,1


In [36]:
print(sorted(train['범죄발생지'].unique()))
print(sorted(test['범죄발생지'].unique()))

['공원', '백화점', '병원', '식당', '약국', '은행', '인도', '주거지', '주유소', '주차장', '차도', '편의점', '학교', '호텔/모텔']
['공원', '백화점', '병원', '식당', '약국', '은행', '인도', '주거지', '주유소', '주차장', '차도', '편의점', '학교', '호텔/모텔']


In [37]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(42) # Seed 고정

In [38]:
x_train = train.drop(['ID'], axis = 1)
# y_train = train['TARGET']

x_test = test.drop('ID', axis = 1)

In [39]:
ordinal_features = ['요일', '범죄발생지']

for feature in ordinal_features:
    le = LabelEncoder()
    le = le.fit(x_train[feature])
    x_train[feature] = le.transform(x_train[feature])

    # x_train데이터에서 존재하지 않았던 값이 x_test 데이터에 존재할 수도 있습니다.
    # 따라서 x_test 데이터를 바로 변형시키지 않고 고윳값을 확인후 x_test 데이터를 변환합니다.
    for label in np.unique(x_test[feature]):
        if label not in le.classes_:
            le.classes_ = np.append(le.classes_, label)
    x_test[feature] = le.transform(x_test[feature])

In [41]:
from pycaret import classification
classification.setup(data=x_train,target='TARGET',
                 remove_outliers=True)

C:\Users\User\anaconda3\envs\git_repo\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  warnings.warn(
C:\Users\User\anaconda3\envs\git_repo\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  warnings.warn(


,Description,Value
0,Session id,1974
1,Target,TARGET
2,Target type,Multiclass
3,Original data shape,"(84406, 19)"
4,Transformed data shape,"(81451, 19)"
5,Transformed train set shape,"(56129, 19)"
6,Transformed test set shape,"(25322, 19)"
7,Numeric features,18
8,Preprocess,True
9,Imputation type,simple


In [42]:
best = classification.compare_models(n_select=5,fold=5)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lightgbm,Light Gradient Boosting Machine,0.5492,0.6990,0.5492,0.5460,0.5389,0.2884,0.2942,0.7840
gbc,Gradient Boosting Classifier,0.5477,0.6970,0.5477,0.5462,0.5373,0.2863,0.2924,5.4480
xgboost,Extreme Gradient Boosting,0.5435,0.6925,0.5435,0.5392,0.5335,0.2801,0.2853,4.0460
ada,Ada Boost Classifier,0.5428,0.6861,0.5428,0.5406,0.5333,0.2809,0.2859,0.8800
catboost,CatBoost Classifier,0.5419,0.6923,0.5419,0.5371,0.5315,0.2773,0.2825,7.1320
rf,Random Forest Classifier,0.5238,0.6739,0.5238,0.5168,0.5133,0.2491,0.2534,2.2140
et,Extra Trees Classifier,0.4997,0.6523,0.4997,0.4909,0.4900,0.2134,0.2163,2.1360
nb,Naive Bayes,0.4687,0.6106,0.4687,0.4645,0.4632,0.1761,0.1774,0.4020
lda,Linear Discriminant Analysis,0.4560,0.6001,0.4560,0.4178,0.3868,0.0935,0.1142,0.4400
ridge,Ridge Classifier,0.4543,0.0000,0.4543,0.3973,0.3703,0.0826,0.1072,0.3800


Processing:   0%|          | 0/71 [00:00<?, ?it/s]

In [47]:
probas = []
for model in best:
    probas.append(model.predict_proba(x_test))

pred = [np.argmax(i) for i in probas[0]] # np.array(probas).mean(axis=0)

In [48]:
sub = pd.read_csv('./sample_submission.csv')
sub.set_index('ID',inplace=True)
sub['TARGET'] = pred
sub.to_csv('./answer.csv')